##import drive and kaggle token

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!cd drive/My\ Drive

# `cd` to the location of your jupyter notebook
!cd Colab\ Notebooks

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/bin/bash: line 0: cd: Colab Notebooks: No such file or directory


In [3]:
# To import kaggle datasets!pip install kaggle

# import colab libraries
from google.colab import files

# import kaggle json to connect to kaggle user account to download datsets
files.upload()

# see if kaggle json exists
!ls -lha kaggle.json'

# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

# download our dataset
!kaggle competitions download -c cat-in-the-dat --force

!unzip "*.csv.zip" 

Saving kaggle.json to kaggle.json
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file
 73% 9.00M/12.3M [00:01<00:00, 9.30MB/s]
100% 12.3M/12.3M [00:01<00:00, 11.6MB/s]
 61% 5.00M/8.15M [00:00<00:00, 12.6MB/s]
100% 8.15M/8.15M [00:00<00:00, 18.4MB/s]
  0% 0.00/437k [00:00<?, ?B/s]
100% 437k/437k [00:00<00:00, 145MB/s]
Archive:  train.csv.zip
  inflating: train.csv               

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

3 archives were successfully processed.


In [1]:
!ls

drive	     sample_submission.csv	test.csv.zip
kaggle.json  sample_submission.csv.zip	train.csv
sample_data  test.csv			train.csv.zip


In [0]:
#!pip install catboost

#Import the libraries and csv

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import xgboost as XGB
import lightgbm as LGB  
import pickle  
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
import scipy
from itertools import product
from sklearn.svm import SVC
import keras
import tensorflow as tf
from mlxtend.classifier import StackingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB, ComplementNB 


Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [4]:
train = pd.read_csv('train.csv')
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [5]:
test = pd.read_csv('test.csv')

test.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,Piano,0870b0a5d,9ceb19dd6,530f8ecc3,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
1,300001,0,0,0,T,N,Red,Square,Lion,Canada,Piano,a5c276589,1ad744242,12e6161c9,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
2,300002,1,0,1,F,Y,Blue,Square,Dog,China,Piano,568550f04,1fe17a1fd,27d6df03f,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12
3,300003,0,0,1,T,Y,Red,Star,Cat,China,Piano,c5725677e,a6542cec0,30c63bd0c,0b6ec68ff,b5de3dcc4,1,Contributor,Lava Hot,b,Q,ke,2,3
4,300004,0,1,1,F,N,Red,Trapezoid,Dog,China,Piano,e70a6270d,97b6a3518,a42386065,f91f3b1ee,967cfa9c9,3,Grandmaster,Lava Hot,l,W,qK,4,11


In [6]:
train_size=train.shape[0]
test_size=test.shape[0]
print('Train size = ' + str(train_size))
print('Test size = ' + str(test_size))

Train size = 300000
Test size = 200000


In [7]:
y_train_tmp=train['target']

whole_dataset=pd.concat([train.drop('target',axis=1), test], axis=0)
whole_dataset.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8


In [8]:
whole_dataset.isnull().sum()

id       0
bin_0    0
bin_1    0
bin_2    0
bin_3    0
bin_4    0
nom_0    0
nom_1    0
nom_2    0
nom_3    0
nom_4    0
nom_5    0
nom_6    0
nom_7    0
nom_8    0
nom_9    0
ord_0    0
ord_1    0
ord_2    0
ord_3    0
ord_4    0
ord_5    0
day      0
month    0
dtype: int64

##Data Preparation

In [9]:
whole_dataset = whole_dataset.drop(['id', ], axis=1)
whole_dataset.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8


####One Hot Encoder



In [0]:
OneHot=OneHotEncoder(drop='first', sparse=True)
OneHot.fit(whole_dataset)
OHtrial_train_tmp=OneHot.transform(whole_dataset.iloc[:train_size,:])
OHtrial_test=OneHot.transform(whole_dataset.iloc[train_size:,:])


In [0]:
OHtrial_train, OHtrial_val, y_train, y_val = train_test_split(OHtrial_train_tmp, y_train_tmp, test_size=0.1, random_state=9)
OHtrial_train=OHtrial_train.astype(float)
OHtrial_val=OHtrial_val.astype(float)

##Model

###Random Forest


In [0]:
rfc=RandomForestClassifier(random_state=42)

In [13]:
'''params={'n_estimators': [100,200,500],
    'max_features': [.2,.3,.5,.6],
    'max_depth':[1 ,3 ,5]
    }
'''


"params={'n_estimators': [100,200,500],\n    'max_features': [.2,.3,.5,.6],\n    'max_depth':[1 ,3 ,5]\n    }\n"

In [14]:
'''arfc_gridsearchcv=GridSearchCV(rfc, params, cv=4, scoring=make_scorer(roc_auc_score), n_jobs=-1, verbose=4)
rfc_gridsearchcv.fit(OHtrial_train, y_train,)
print("Best score is " + str(rfc_gridsearchcv.best_score_) + " with parameters:")
print(rfc_gridsearchcv.best_params_)
'''

'''
Best score is 0.523748348402897 with parameters:
{'max_depth': 5, 'max_features': 0.6, 'n_estimators': 200}
'''

"\nBest score is 0.523748348402897 with parameters:\n{'max_depth': 5, 'max_features': 0.6, 'n_estimators': 200}\n"

In [15]:
rfc=RandomForestClassifier(max_depth=5, n_estimators=200, max_features=0.6,random_state=42,n_jobs=-1,verbose=1)
rfc.fit(OHtrial_train, y_train)

y_train_pred_rfc=rfc.predict_proba(OHtrial_train)[:,1]
y_val_pred_rfc=rfc.predict_proba(OHtrial_val)[:,1]
y_test_pred_rfc=rfc.predict_proba(OHtrial_test)[:,1]
print("Training auc : ",roc_auc_score(y_train, y_train_pred_rfc))
print("Val auc : ",roc_auc_score(y_val, y_val_pred_rfc))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  1.4min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.1s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    2.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 192 tasks   

Training auc :  0.6526666378084609
Val auc :  0.642487318333132


###LGBM


#### Parameter tunning

In [16]:
'''max_depth_list=[1,2]
learning_rate_list=[.45 ,.5 ,0.55 ,0.6,]
feature_fraction_list=[.1,.2,.3]

max_depth_done=[]
learning_rate_done=[]
feature_fraction_done=[]
auc_mean=[]
auc_std=[]'''

'max_depth_list=[1,2]\nlearning_rate_list=[.45 ,.5 ,0.55 ,0.6,]\nfeature_fraction_list=[.1,.2,.3]\n\nmax_depth_done=[]\nlearning_rate_done=[]\nfeature_fraction_done=[]\nauc_mean=[]\nauc_std=[]'

In [17]:
'''for max_depth in max_depth_list:
  for learning_rate in learning_rate_list:
    for feature_fraction in feature_fraction_list:
      
      lgb_train = LGB.Dataset(OHtrial_train, y_train)  
      lgb_eval = LGB.Dataset(OHtrial_val, y_val, reference=lgb_train) 
      params = {  
          'boosting_type': 'gbdt',  
          'objective': 'binary',  
          'metric': {
              #'binary_logloss', 
                      'auc'},  
          #'num_leaves':450,  
          'max_depth': max_depth,  
          #'min_data_in_leaf': 150,  
          'learning_rate': learning_rate,  
          'feature_fraction': feature_fraction,  
          #'bagging_fraction': bagging_fraction,  
          #'bagging_freq': 10,  
          #'lambda_l1': 0,    
          #'lambda_l2': 0, 
          #'min_gain_to_split': 0.1,  
          'verbose': 0,  
          'is_unbalance': True  
      }  
      cv_result=LGB.cv(params,
          lgb_train, 
          num_boost_round=10000,  
          early_stopping_rounds=500,
          nfold=3,
          metrics='auc')

      auc_mean.append(cv_result['auc-mean'][-1])
      auc_std.append(cv_result['auc-stdv'][-1])
      max_depth_done.append(max_depth)
      learning_rate_done.append(learning_rate)
      feature_fraction_done.append(feature_fraction)'''
      

"for max_depth in max_depth_list:\n  for learning_rate in learning_rate_list:\n    for feature_fraction in feature_fraction_list:\n      \n      lgb_train = LGB.Dataset(OHtrial_train, y_train)  \n      lgb_eval = LGB.Dataset(OHtrial_val, y_val, reference=lgb_train) \n      params = {  \n          'boosting_type': 'gbdt',  \n          'objective': 'binary',  \n          'metric': {\n              #'binary_logloss', \n                      'auc'},  \n          #'num_leaves':450,  \n          'max_depth': max_depth,  \n          #'min_data_in_leaf': 150,  \n          'learning_rate': learning_rate,  \n          'feature_fraction': feature_fraction,  \n          #'bagging_fraction': bagging_fraction,  \n          #'bagging_freq': 10,  \n          #'lambda_l1': 0,    \n          #'lambda_l2': 0, \n          #'min_gain_to_split': 0.1,  \n          'verbose': 0,  \n          'is_unbalance': True  \n      }  \n      cv_result=LGB.cv(params,\n          lgb_train, \n          num_boost_round=100

In [18]:
'''auc_df=pd.DataFrame(
    list(zip(max_depth_done, learning_rate_done, feature_fraction_done,  auc_mean, auc_std)),
    columns=['max_depth', 'learning_rate', 'feature_fraction', 'auc_mean', 'auc_std'],
)
auc_df.sort_values()'''

#best depth=1,lr=0.45, feature=0.2 

"auc_df=pd.DataFrame(\n    list(zip(max_depth_done, learning_rate_done, feature_fraction_done,  auc_mean, auc_std)),\n    columns=['max_depth', 'learning_rate', 'feature_fraction', 'auc_mean', 'auc_std'],\n)\nauc_df.sort_values()"

In [19]:
'''result=LGB.cv(params,
       lgb_train, 
       num_boost_round=10,  
       early_stopping_rounds=500,
       nfold=3,
       metrics='auc')
print(result['auc-mean'][-1])
print(result['auc-stdv'][-1])'''

"result=LGB.cv(params,\n       lgb_train, \n       num_boost_round=10,  \n       early_stopping_rounds=500,\n       nfold=3,\n       metrics='auc')\nprint(result['auc-mean'][-1])\nprint(result['auc-stdv'][-1])"

#### Using the best parameter found in tunning


In [0]:

lgb_train = LGB.Dataset(OHtrial_train, y_train)  
lgb_eval = LGB.Dataset(OHtrial_val, y_val, reference=lgb_train) 
params = {  
    'boosting_type': 'gbdt',  
    'objective': 'binary',  
    'metric': {
        #'binary_logloss', 
               'auc'},  
    #'num_leaves':450,  
    'max_depth': 1,  
    #'min_data_in_leaf': 1000,  
    'learning_rate': 0.45,  
    'feature_fraction': 0.2,  
    #'bagging_fraction': 0.2,  
    #'bagging_freq': 10,  
    #'lambda_l1': 0,    
    #'lambda_l2': 0, 
    #'min_gain_to_split': 0.1,
    'is_unbalance': True  
}  

In [21]:
lgb = LGB.train(params,  
          lgb_train,  
          num_boost_round=20000,  
          valid_sets=[lgb_train, lgb_eval, ],  
          early_stopping_rounds=500,
          verbose_eval=100
          ) 

Training until validation scores don't improve for 500 rounds.
[100]	training's auc: 0.727683	valid_1's auc: 0.720199
[200]	training's auc: 0.74804	valid_1's auc: 0.74141
[300]	training's auc: 0.75791	valid_1's auc: 0.750526
[400]	training's auc: 0.764297	valid_1's auc: 0.757753
[500]	training's auc: 0.76869	valid_1's auc: 0.761114
[600]	training's auc: 0.772067	valid_1's auc: 0.764221
[700]	training's auc: 0.775412	valid_1's auc: 0.766712
[800]	training's auc: 0.777899	valid_1's auc: 0.768567
[900]	training's auc: 0.780251	valid_1's auc: 0.770554
[1000]	training's auc: 0.782462	valid_1's auc: 0.772565
[1100]	training's auc: 0.784214	valid_1's auc: 0.77376
[1200]	training's auc: 0.786026	valid_1's auc: 0.775072
[1300]	training's auc: 0.787379	valid_1's auc: 0.775962
[1400]	training's auc: 0.789034	valid_1's auc: 0.777298
[1500]	training's auc: 0.790469	valid_1's auc: 0.778207
[1600]	training's auc: 0.791844	valid_1's auc: 0.778673
[1700]	training's auc: 0.792805	valid_1's auc: 0.779226

In [22]:
y_train_pred_lgb=lgb.predict(OHtrial_train, num_iteration=lgb.best_iteration)
y_val_pred_lgb=lgb.predict(OHtrial_val, num_iteration=lgb.best_iteration)
y_test_pred_lgb=lgb.predict(OHtrial_test, num_iteration=lgb.best_iteration) 
print("Training auc : ",roc_auc_score(y_train, y_train_pred_lgb))
print("Val auc : ",roc_auc_score(y_val, y_val_pred_lgb))

Training auc :  0.8160321022194943
Val auc :  0.7912257540554954


###Logistic Regression


#### Parameter tunning

In [0]:
logreg=LogisticRegression(random_state=123)

In [24]:
'''params_logreg={'penalty': ['l1', 'l2'],
    'C': [.05, .1, .5, 1.0, 2.0, 5.0, 10.0],
    'max_iter': [20, 50, 100, 200, 500]
    }'''

"params_logreg={'penalty': ['l1', 'l2'],\n    'C': [.05, .1, .5, 1.0, 2.0, 5.0, 10.0],\n    'max_iter': [20, 50, 100, 200, 500]\n    }"

In [25]:
'''
gridsearchcv_logreg=GridSearchCV(logreg, params_logreg, scoring=make_scorer(roc_auc_score), n_jobs=-1, cv=4, verbose=10)
gridsearchcv_logreg.fit(OHtrial_train, y_train)
print("The best score is ", gridsearchcv_logreg.best_score_, ' with parameters:')
print(gridsearchcv_logreg.best_params_)
logreg=gridsearchcv_logreg.best_estimator_
'''

'\ngridsearchcv_logreg=GridSearchCV(logreg, params_logreg, scoring=make_scorer(roc_auc_score), n_jobs=-1, cv=4, verbose=10)\ngridsearchcv_logreg.fit(OHtrial_train, y_train)\nprint("The best score is ", gridsearchcv_logreg.best_score_, \' with parameters:\')\nprint(gridsearchcv_logreg.best_params_)\nlogreg=gridsearchcv_logreg.best_estimator_\n'

#### Using the best parameter found in tunning



In [0]:
logreg=LogisticRegression(C=0.1, class_weight='balanced',penalty='l2',max_iter=20, n_jobs=-1, solver='lbfgs', random_state=123)

In [27]:
logreg.fit(OHtrial_train, y_train)
y_train_pred_logreg=logreg.predict_proba(OHtrial_train)[:, 1]
y_val_pred_logreg=logreg.predict_proba(OHtrial_val,)[:, 1]
print("Training auc : ",roc_auc_score(y_train, y_train_pred_logreg))
print("Val auc : ",roc_auc_score(y_val, y_val_pred_logreg))

Training auc :  0.8031688255617003
Val auc :  0.7881070891375649


In [0]:
y_test_pred_logreg=logreg.predict_proba(OHtrial_test,)[:, 1]

###XGBoost

In [29]:
#OHtrial_train=OHtrial_train.tocsr()
train_data=XGB.DMatrix(OHtrial_train, label=y_train)
val_data=XGB.DMatrix(OHtrial_val, label=y_val)
param={"objective": "binary:logistic", 
    "eval_metric": "auc", 
    'max_depth': 6, #best 6
    'eta':0.1, #best 0.1
    'colsample_bytree':0.4, # best 0.4
    'min_data_in_leaf': 1000, #best None
    'subsample':0.19, #best 0.2
    #'reg_alpha':0.5 #best None
    #'njobs':-1 #best None
    'random_state': 42,
    }
model=XGB.train(param, 
                dtrain =train_data, 
                num_boost_round=10000 #best 5000
                ,evals=[(train_data, 'train'), (val_data,'val')], 
                early_stopping_rounds=50, #best 50
               )
y_train_pred=model.predict(XGB.DMatrix(OHtrial_train))
y_val_pred=model.predict(XGB.DMatrix(OHtrial_val))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-auc:0.627044	val-auc:0.620278
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 50 rounds.
[1]	train-auc:0.65052	val-auc:0.641323
[2]	train-auc:0.662793	val-auc:0.653403
[3]	train-auc:0.669002	val-auc:0.657457
[4]	train-auc:0.66912	val-auc:0.657388
[5]	train-auc:0.674098	val-auc:0.662177
[6]	train-auc:0.680172	val-auc:0.667816
[7]	train-auc:0.682434	val-auc:0.66997
[8]	train-auc:0.686039	val-auc:0.674964
[9]	train-auc:0.684696	val-auc:0.674318
[10]	train-auc:0.685179	val-auc:0.674399
[11]	train-auc:0.685608	val-auc:0.67457
[12]	train-auc:0.686339	val-auc:0.675314
[13]	train-auc:0.690563	val-auc:0.679044
[14]	train-auc:0.692453	val-auc:0.680327
[15]	train-auc:0.694642	val-auc:0.682532
[16]	train-auc:0.696227	val-auc:0.683904
[17]	train-auc:0.698607	val-auc:0.685524
[18]	train-auc:0.699912	val-auc:0.686649
[19]	train-auc:0.70116	val-auc:0.687915
[20]	train-auc:0.701578	val-auc:0.687586
[21]	train-auc:

In [30]:
print("Training score : ", roc_auc_score(y_train, y_train_pred))
print("Validation score : ", roc_auc_score(y_val, y_val_pred))

Training score :  0.8262596309336081
Validation score :  0.7802246306345371


In [0]:
y_test_pred_xgb=model.predict(XGB.DMatrix(OHtrial_test))

### DNN

In [0]:
def auc_roc(y_true, y_pred):
  return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [33]:
DNN=keras.models.Sequential()
DNN.add(keras.layers.Dense(10, input_shape=(16529, ), activation='relu'))
DNN.add(keras.layers.Dropout(0.001))
DNN.add(keras.layers.BatchNormalization())
DNN.add(keras.layers.Dense(1, activation='sigmoid'))

DNN.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=[auc_roc])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [34]:
DNN.fit(OHtrial_train, y_train, 
    validation_data=(OHtrial_val, y_val), 
    epochs=100, 
    batch_size=3000, 
    callbacks=[
          keras.callbacks.ReduceLROnPlateau(monitor='val_loss'), 
          keras.callbacks.EarlyStopping(monitor='val_loss')
          ])




Train on 270000 samples, validate on 30000 samples
Epoch 1/100





270000/270000 [==============================] - 29s 106us/step - loss: 0.6875 - auc_roc: 0.6178 - val_loss: 0.6482 - val_auc_roc: 0.6883
Epoch 2/100
270000/270000 [==============================] - 24s 88us/step - loss: 0.6072 - auc_roc: 0.7472 - val_loss: 0.6152 - val_auc_roc: 0.7526
Epoch 3/100
270000/270000 [==============================] - 24s 88us/step - loss: 0.5693 - auc_roc: 0.7909 - val_loss: 0.5779 - val_auc_roc: 0.7727
Epoch 4/100
270000/270000 [==============================] - 24s 88us/step - loss: 0.5457 - auc_roc: 0.8107 - val_loss: 0.5639 - val_auc_roc: 0.7799
Epoch 5/100
270000/270000 [==============================] - 24s 88us/step - loss: 0.5281 - auc_roc: 0.8223 - val_loss: 0.5535 - val_auc_roc: 0.7827
Epoch 6/100
270000/270000 [==============================] - 24s 87us/step - loss: 0.5134 - auc_roc: 0.8305 - val_loss: 0.5401 - val_auc_roc: 0.7835
Epoch 7/100
270000/270000 [===================

##Submission

In [0]:
y_test_pred = (y_test_pred_lgb+y_test_pred_logreg)/2
submission=pd.DataFrame({'id': np.arange(300000, 500000,1), 'target':y_test_pred})
submission.to_csv('submission.csv', index=False)


In [37]:
pd.read_csv('submission.csv').head()


,id,target
0,300000,0.463576
1,300001,0.844508
2,300002,0.258495
3,300003,0.587108
4,300004,0.905055


In [38]:
!kaggle competitions submit -c cat-in-the-dat -f submission.csv -m "Message"

100% 5.01M/5.01M [00:03<00:00, 1.49MB/s]
Successfully submitted to Categorical Feature Encoding Challenge